In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from uuid import uuid4
from pymongo import MongoClient
import urllib

In [34]:
df = pd.read_csv("../static/data/itc.csv")
df = df[df["sub_cate_report"] != "TIKI-Ngon"]
df["product_id"] = df["product_id"].astype("int64")
df["sku"] = df["sku"].astype("int64")
df = df.head(100)

In [30]:
df["url"] = df["product_id"].apply(lambda x: "https://tiki.vn/p/" + str(x))

In [31]:
df.head()

,product_id,sku,product_name,cate_report,sub_cate_report,cf_orders,url
0,32033727,8980713037420,Apple iPhone 11 64GB Xanh Lục,Phones - Tablets,Phones - Tablets,196,https://tiki.vn/p/32033727
1,32033721,5679549432792,Apple iPhone 11 64GB Trắng,Phones - Tablets,Phones - Tablets,194,https://tiki.vn/p/32033721
2,88255888,7196209362779,Điện Thoại Oppo A94 (8GB/128G) - Hàng Chính Hã...,Phones - Tablets,Phones - Tablets,188,https://tiki.vn/p/88255888
3,113569121,1297769243041,Điện Thoại Oppo Reno 6Z 5G (8GB/128G) - Hàng C...,Phones - Tablets,Phones - Tablets,112,https://tiki.vn/p/113569121
4,123554562,8632767284726,Apple iPhone 13 Pro Max 128GB Xanh Sierra,Phones - Tablets,Phones - Tablets,109,https://tiki.vn/p/123554562


In [32]:
url_eg = df.iloc[0,-1]
url_eg

'https://tiki.vn/p/32033727'

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url=url_eg)

[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:00<00:00, 8.73MB/s]
/var/folders/4y/vhc9jlvn1v7b0h3qkld4mk0m0000gn/T/ipykernel_4833/261311818.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [10]:
uri = "mongodb+srv://vietthangc1:"+urllib.parse.quote_plus('f2bdx@*-uLAZz!f')+"@cluster0.le7ea.mongodb.net/test"
db = MongoClient(uri)["ecom"]

In [35]:
i = 0
list_insert = []

for row in df.iterrows():
  item = row[1]
  i += 1

  brand = None
  price = None
  seller = None

  driver.get(url="https://tiki.vn/p/" + str(item["product_id"]))
  time.sleep(0.5)
  html = BeautifulSoup(driver.page_source)

  try:
    brand_div = html.find("div", class_ = "brand")
    brand = brand_div.find("span", class_ = "brand-and-author").find("a").get_text()
  except:
    pass

  try:
    price_div = html.find("div", class_ = "price-and-icon")

    flash_sale_div = price_div.find_all("div", class_ = "flash-sale-price")
    if len(flash_sale_div) > 0:
      price = flash_sale_div[0].find("span").get_text()
    else:
      price = price_div.find("div", class_ = "product-price__current-price").get_text()
    price = price.replace(".","").replace(" ₫", "")
    price = int(price)
  except:
    pass

  try: 
    img_div = html.find("div", class_ = "review-images")
    imgs = img_div.find_all("img")
    img_srcs = []
    for img in imgs:
      img_src = img.get("src")
      webs = requests.get(img_src)
      filename = uuid4().hex + "." + img_src.split(".")[-1]
      open("../static/img/product_img/" + filename, "wb").write(webs.content)
      img_srcs.append(filename)
  except:
    pass

  try:
    content_div = html.find_all("div", class_ = "content")[-1]
    lst_e = content_div.find("div").find("div").find_all()
    html_desc = ""
    for e in lst_e:
      html_desc += str(e)
  except:
    pass

  try:
    seller_div = html.find("div", class_ = "seller-info")
    seller = seller_div.find("span", class_ = "seller-name").find("span").get_text()
  except:
    pass

  dict = {
    "_id": uuid4().hex,
    "product_id": item["product_id"],
    "product_name": item["product_name"],
    "cate_report": item["cate_report"],
    "sub_cate_report": item["sub_cate_report"],
    "brand": brand,
    "price": price,
    "seller": seller,
    "img_srcs": img_srcs,
    "description": html_desc
  }

  list_insert.append(dict)


db.product.insert_many(list_insert)

In [36]:
df = pd.read_csv("../static/data/lifestyle.csv")
df = df[df["sub_cate_report"] != "TIKI-Ngon"]
df["product_id"] = df["product_id"].astype("int64")
df["sku"] = df["sku"].astype("int64")
df = df.head(100)

In [37]:
i = 0
list_insert = []

for row in df.iterrows():
  item = row[1]
  i += 1

  brand = None
  price = None
  seller = None

  driver.get(url="https://tiki.vn/p/" + str(item["product_id"]))
  time.sleep(0.5)
  html = BeautifulSoup(driver.page_source)

  try:
    brand_div = html.find("div", class_ = "brand")
    brand = brand_div.find("span", class_ = "brand-and-author").find("a").get_text()
  except:
    pass

  try:
    price_div = html.find("div", class_ = "price-and-icon")

    flash_sale_div = price_div.find_all("div", class_ = "flash-sale-price")
    if len(flash_sale_div) > 0:
      price = flash_sale_div[0].find("span").get_text()
    else:
      price = price_div.find("div", class_ = "product-price__current-price").get_text()
    price = price.replace(".","").replace(" ₫", "")
    price = int(price)
  except:
    pass

  try: 
    img_div = html.find("div", class_ = "review-images")
    imgs = img_div.find_all("img")
    img_srcs = []
    for img in imgs:
      img_src = img.get("src")
      webs = requests.get(img_src)
      filename = uuid4().hex + "." + img_src.split(".")[-1]
      open("../static/img/product_img/" + filename, "wb").write(webs.content)
      img_srcs.append(filename)
  except:
    pass

  try:
    content_div = html.find_all("div", class_ = "content")[-1]
    lst_e = content_div.find("div").find("div").find_all()
    html_desc = ""
    for e in lst_e:
      html_desc += str(e)
  except:
    pass

  try:
    seller_div = html.find("div", class_ = "seller-info")
    seller = seller_div.find("span", class_ = "seller-name").find("span").get_text()
  except:
    pass

  dict = {
    "_id": uuid4().hex,
    "product_id": item["product_id"],
    "product_name": item["product_name"],
    "cate_report": item["cate_report"],
    "sub_cate_report": item["sub_cate_report"],
    "brand": brand,
    "price": price,
    "seller": seller,
    "img_srcs": img_srcs,
    "description": html_desc
  }

  list_insert.append(dict)


db.product.insert_many(list_insert)

In [38]:
df = pd.read_csv("../static/data/homeappliances.csv")
df = df[df["sub_cate_report"] != "TIKI-Ngon"]
df["product_id"] = df["product_id"].astype("int64")
df["sku"] = df["sku"].astype("int64")
df = df.head(100)

In [39]:
i = 0
list_insert = []

for row in df.iterrows():
  item = row[1]
  i += 1

  brand = None
  price = None
  seller = None

  driver.get(url="https://tiki.vn/p/" + str(item["product_id"]))
  time.sleep(0.5)
  html = BeautifulSoup(driver.page_source)

  try:
    brand_div = html.find("div", class_ = "brand")
    brand = brand_div.find("span", class_ = "brand-and-author").find("a").get_text()
  except:
    pass

  try:
    price_div = html.find("div", class_ = "price-and-icon")

    flash_sale_div = price_div.find_all("div", class_ = "flash-sale-price")
    if len(flash_sale_div) > 0:
      price = flash_sale_div[0].find("span").get_text()
    else:
      price = price_div.find("div", class_ = "product-price__current-price").get_text()
    price = price.replace(".","").replace(" ₫", "")
    price = int(price)
  except:
    pass

  try: 
    img_div = html.find("div", class_ = "review-images")
    imgs = img_div.find_all("img")
    img_srcs = []
    for img in imgs:
      img_src = img.get("src")
      webs = requests.get(img_src)
      filename = uuid4().hex + "." + img_src.split(".")[-1]
      open("../static/img/product_img/" + filename, "wb").write(webs.content)
      img_srcs.append(filename)
  except:
    pass

  try:
    content_div = html.find_all("div", class_ = "content")[-1]
    lst_e = content_div.find("div").find("div").find_all()
    html_desc = ""
    for e in lst_e:
      html_desc += str(e)
  except:
    pass

  try:
    seller_div = html.find("div", class_ = "seller-info")
    seller = seller_div.find("span", class_ = "seller-name").find("span").get_text()
  except:
    pass

  dict = {
    "_id": uuid4().hex,
    "product_id": item["product_id"],
    "product_name": item["product_name"],
    "cate_report": item["cate_report"],
    "sub_cate_report": item["sub_cate_report"],
    "brand": brand,
    "price": price,
    "seller": seller,
    "img_srcs": img_srcs,
    "description": html_desc
  }

  list_insert.append(dict)


db.product.insert_many(list_insert)

In [40]:
df = pd.read_csv("../static/data/cg.csv")
df = df[df["sub_cate_report"] != "TIKI-Ngon"]
df["product_id"] = df["product_id"].astype("int64")
df["sku"] = df["sku"].astype("int64")
df = df.head(100)

In [41]:
i = 0
list_insert = []

for row in df.iterrows():
  item = row[1]
  i += 1

  brand = None
  price = None
  seller = None

  driver.get(url="https://tiki.vn/p/" + str(item["product_id"]))
  time.sleep(0.5)
  html = BeautifulSoup(driver.page_source)

  try:
    brand_div = html.find("div", class_ = "brand")
    brand = brand_div.find("span", class_ = "brand-and-author").find("a").get_text()
  except:
    pass

  try:
    price_div = html.find("div", class_ = "price-and-icon")

    flash_sale_div = price_div.find_all("div", class_ = "flash-sale-price")
    if len(flash_sale_div) > 0:
      price = flash_sale_div[0].find("span").get_text()
    else:
      price = price_div.find("div", class_ = "product-price__current-price").get_text()
    price = price.replace(".","").replace(" ₫", "")
    price = int(price)
  except:
    pass

  try: 
    img_div = html.find("div", class_ = "review-images")
    imgs = img_div.find_all("img")
    img_srcs = []
    for img in imgs:
      img_src = img.get("src")
      webs = requests.get(img_src)
      filename = uuid4().hex + "." + img_src.split(".")[-1]
      open("../static/img/product_img/" + filename, "wb").write(webs.content)
      img_srcs.append(filename)
  except:
    pass

  try:
    content_div = html.find_all("div", class_ = "content")[-1]
    lst_e = content_div.find("div").find("div").find_all()
    html_desc = ""
    for e in lst_e:
      html_desc += str(e)
  except:
    pass

  try:
    seller_div = html.find("div", class_ = "seller-info")
    seller = seller_div.find("span", class_ = "seller-name").find("span").get_text()
  except:
    pass

  dict = {
    "_id": uuid4().hex,
    "product_id": item["product_id"],
    "product_name": item["product_name"],
    "cate_report": item["cate_report"],
    "sub_cate_report": item["sub_cate_report"],
    "brand": brand,
    "price": price,
    "seller": seller,
    "img_srcs": img_srcs,
    "description": html_desc
  }

  list_insert.append(dict)


db.product.insert_many(list_insert)